In [2]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://www.newegg.com/p/N82E16834840524'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [3]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [10]:
data = json_data_list[2]

In [13]:
data

{'@context': 'http://schema.org/',
 '@type': 'Product',
 'offers': {'@type': 'Offer',
  'availability': 'http://schema.org/InStock',
  'price': '659.99',
  'priceCurrency': 'USD',
  'url': 'https://www.newegg.com/lenovo-ideapad-5-2-in-1-16iru9-83du0005us-16-0-intel-core-5-120u-512-gb-pcie-ssd-8gb-memory/p/N82E16834840524'},
 'description': 'Lenovo IdeaPad 5 2-in-1 16IRU9 Intel Core 5 120U 8GB Memory 512 GB PCIe SSD Intel Graphics 16.0" Touchscreen 1920 x 1200 Convertible 2 in 1 Laptop Windows 11 Home 64-bit 83DU0005US',
 'name': 'Lenovo IdeaPad 5 2-in-1 16IRU9 Intel Core 5 120U 8GB Memory 512 GB PCIe SSD Intel Graphics 16.0" Touchscreen 1920 x 1200 Convertible 2 in 1 Laptop Windows 11 Home 64-bit 83DU0005US',
 'sku': 'N82E16834840524',
 'brand': 'Lenovo',
 'mpn': '83DU0005US',
 'gtin12': '197531779267',
 'Model': '83DU0005US',
 'image': 'https://c1.neweggimages.com/ProductImage/A0ZXD24041017RBOG9D.jpg',
 'width': '11 in',
 'height': '3 in',
 'weight': '4.8 oz',
 'itemCondition': 'NewCo

In [15]:
extracted_data = {
    'name': data.get('name'),
    'sku': data.get('sku'),
    'gtin12': data.get('gtin12'),
    'description': data.get('description'),
    'brand': data.get('brand'),
    'mpn': data.get('mpn'),
    'Model': data.get('Model'),
    'price': data['offers'].get('price'),
    'priceCurrency': data['offers'].get('priceCurrency'),
    'availability': data['offers'].get('availability'),
    'image': data.get('image'),
    'height': data.get('height'),
    'width': data.get('width'),
    'length': None,  # length not provided in the JSON
    'itemcondition': data.get('itemCondition')
}

In [16]:
print(json.dumps(extracted_data, indent=4))

{
    "name": "Lenovo IdeaPad 5 2-in-1 16IRU9 Intel Core 5 120U 8GB Memory 512 GB PCIe SSD Intel Graphics 16.0\" Touchscreen 1920 x 1200 Convertible 2 in 1 Laptop Windows 11 Home 64-bit 83DU0005US",
    "sku": "N82E16834840524",
    "gtin12": "197531779267",
    "description": "Lenovo IdeaPad 5 2-in-1 16IRU9 Intel Core 5 120U 8GB Memory 512 GB PCIe SSD Intel Graphics 16.0\" Touchscreen 1920 x 1200 Convertible 2 in 1 Laptop Windows 11 Home 64-bit 83DU0005US",
    "brand": "Lenovo",
    "mpn": "83DU0005US",
    "Model": "83DU0005US",
    "price": "659.99",
    "priceCurrency": "USD",
    "availability": "http://schema.org/InStock",
    "image": "https://c1.neweggimages.com/ProductImage/A0ZXD24041017RBOG9D.jpg",
    "height": "3 in",
    "width": "11 in",
    "length": null,
    "itemcondition": "NewCondition"
}
